### 从.mat文件中读入数据

In [1]:
import scipy
import numpy as np

In [2]:
data=scipy.io.loadmat('DBPB.mat')#读入由matlab处理好的样本特征向量

In [3]:
data['DBPB1'][0].shape#每个样本94个特征

(94,)

In [4]:
data['DBPB1'].shape#第一类样本数量为9279

(9279, 94)

### 先将数据读入到矩阵中便于处理

In [5]:
data['DBPB1']

array([[0.00172604, 0.00379138, 0.00296589, ..., 0.00116799, 0.00091695,
        0.00525093],
       [0.00352267, 0.00097731, 0.00296935, ..., 0.00847652, 0.00462204,
        0.00282324],
       [0.00339284, 0.00166496, 0.00277194, ..., 0.00288942, 0.00499452,
        0.00355836],
       ...,
       [0.00799746, 0.00392692, 0.00227704, ..., 0.00519247, 0.00426208,
        0.00183095],
       [0.00149238, 0.00276466, 0.00239477, ..., 0.00100603, 0.00415174,
        0.00158   ],
       [0.00478357, 0.00589117, 0.006869  , ..., 0.00381122, 0.01051007,
        0.01011765]])

In [15]:
d_1=np.array(data['DBPB1'])
d_2=np.array(data['DBPB2'])
d_3=np.array(data['DBPB3'])
d_4=np.array(data['DBPB4'])
d_5=np.array(data['DBPB5'])
d_6=np.array(data['DBPB6'])
d_7=np.array(data['DBPB7'])
d_8=np.array(data['DBPB8'])
d_9=np.array(data['DBPB9'])
d_10=np.array(data['DBPB10'])
d_11=np.array(data['DBPB11'])

### 接下来进行数据平衡

In [9]:
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import OneSidedSelection

#### 先进行下采样

In [19]:
x=np.concatenate((d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11),axis=0)
x.shape

(12983, 94)

In [67]:
for i in range(1,12):
    exec(f'y_{i}=np.full(d_{i}.shape[0],i)');

In [68]:
y_dir=[y_1,y_2,y_3,y_4,y_5,y_6,y_7,y_8,y_9,y_10,y_11]

In [70]:
y_dir[0].shape

(9279,)

In [71]:
y=np.concatenate((y_dir),axis=0)
y.shape

(12983,)

In [118]:
def ratio_multiplier(y):
    from collections import Counter

    multiplier = {1: 0.334}
    target_stats = Counter(y)
    for key, value in target_stats.items():
        if key in multiplier:
            target_stats[key] = int(value * multiplier[key])
    return target_stats

ratio_multiplier(y)

Counter({1: 3099,
         2: 710,
         3: 600,
         5: 561,
         4: 454,
         10: 454,
         7: 360,
         6: 251,
         9: 153,
         8: 88,
         11: 73})

In [103]:
#sa_st={0:3100,1:710,2:600,3:454,4:561,5:251,6:360,7:88,8:153,9:454,10:73}
#sa_st={1:3100,2:710,3:600,4:454,5:561,6:251,7:360,8:88,9:153,10:454,11:73}
#该算法在imbalanced中属于清除算法，清除策略应用字典表示目标类
sa_st=[1]
oss=OneSidedSelection(sampling_strategy=sa_st)
x_resampled,y_resampled=oss.fit_resample(x,y)

In [107]:
for i in range(1,12):
    print(sum(y_resampled==i))
for i in range(1,12):
    print(sum(y==i))

8813
710
600
454
561
251
360
88
153
454
73
9279
710
600
454
561
251
360
88
153
454
73


In [129]:
from sklearn.cluster import MiniBatchKMeans
from imblearn.under_sampling import ClusterCentroids
from imblearn.over_sampling import ADASYN

In [130]:
cc=ClusterCentroids(
    sampling_strategy=ratio_multiplier,estimator=MiniBatchKMeans(n_init=1,random_state=0)
)
x_under_resampled,y_under_resampled=cc.fit_resample(x,y)
for i in range(1,12):
    print(sum(y_under_resampled==i))

3099
710
600
454
561
251
360
88
153
454
73


#### 再进行上采样

In [131]:
ada=ADASYN()
x_resampled,y_resampled=ada.fit_resample(x_under_resampled,y_under_resampled)
for i in range(1,12):
    print(sum(y_resampled==i))

3099
2959
3073
3131
2917
3156
3028
3128
3120
3019
3087


重采样完毕

### 构造分类模型